In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.ensemble import BalanceCascade, EasyEnsemble
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import GridSearchCV
import h5py
import keras
from sklearn.utils import class_weight
from keras.utils import to_categorical
from keras.optimizers import *
from keras.regularizers import *
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

KFOLD_SEED = 42

def keras_confusion_matrix(test_labels_1d, predicted_labels):
    m = confusion_matrix(test_labels_1d, predicted_labels)

    print("Confusion Matrix:")
    print("True Negative = %d" % m[0][0])
    print("False Negative = %d" % m[1][0])
    print("True Positive = %d" % m[1][1])
    print("False Positive = %d" % m[0][1])
    

def keras_f1_score(test_labels_1d, predicted_labels):
    f = f1_score(test_labels_1d, predicted_labels)
    print("f1 score = %0.3f" % f)


def print_metrics(test_labels_1d, y_scores, is_train=False):
    predicted_labels = y_scores.argmax(axis=-1)
    if is_train:
        print("---------train---------")
    else:
        print("---------test---------")
    
    keras_confusion_matrix(test_labels_1d, predicted_labels)
    keras_f1_score(test_labels_1d, predicted_labels)
    print("ROC Score = %0.3f" % roc_auc_score(test_labels_1d, predicted_labels))

In [3]:
input_path = '~/data/biddings.csv'
data = pd.read_csv(input_path)
print(data.shape)

train = data[:800000]
test = data[800000:]

sample = train.sample(frac=1)
features = sample.drop('convert', axis=1).values
labels = sample.convert.ravel()
categorical_labels = to_categorical(sample.convert.ravel(), 2)

test_features = test.drop('convert', axis=1).values
test_labels = test.convert.ravel()
categorical_test_labels = to_categorical(test.convert.ravel(), 2)

(1000000, 89)


In [40]:
dt = DecisionTreeClassifier(max_features=0.2, random_state=KFOLD_SEED)
bc = BalanceCascade(estimator=dt, n_max_subset=10, random_state=KFOLD_SEED)

def deep_ensemble(model_fn, model_fit_fn, ensembler, smote=None):
    print("fitting sample")
    X_res, y_res = ensembler.fit_sample(features, labels)
    print(X_res.shape, y_res.shape)
    
    model = model_fn()
    print("training")

    for j, X_train in enumerate(X_res):
        if smote is not None:
            X, y = smote.fit_sample(X_train, y_res[j])
            y = to_categorical(y, 2)
            model = model_fit_fn(model, X, y)
        else:
            y = to_categorical(y_res[j], 2)
            model = model_fit_fn(model, X_train, y)

    predicted_scores = model.predict(test_features, verbose=1)
    print(predicted_scores.shape)
    print_metrics(test_labels, predicted_scores, is_train=False)
    return model
    

## Basic Neural Net

In [24]:
batch_size = 16
epochs = 1

Bare minimum: 0.644 <br>
10 Epochs: 0.655 <br>
With l2(0.01): 0.660

In [26]:
def base_model():
    model = Sequential()
    model.add(Dense(88, kernel_initializer='glorot_uniform', input_shape=(88,)))
    model.add(Dense(2, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(lr=0.01),
                  metrics=['accuracy'])
    return model;

def model_fit(model, X, y):
    model.fit(X, y,
              batch_size=batch_size,
              epochs=10,
#               validation_split=0.2,
              verbose=1)
    return model

model = deep_ensemble(base_model, model_fit, bc)

fitting sample
(10, 3062, 88) (10, 3062, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 88)                7832      
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 178       
Total params: 8,010
Trainable params: 8,010
Non-trainable params: 0
_________________________________________________________________
training
Epoch 1/10
3062/3062 [==============================] - 1s 238us/step - loss: 0.7281 - acc: 0.5993
Epoch 2/10
3062/3062 [==============================] - 0s 113us/step - loss: 0.6580 - acc: 0.6306
Epoch 3/10
3062/3062 [==============================] - 0s 110us/step - loss: 0.6549 - acc: 0.6283
Epoch 4/10
3062/3062 [==============================] - 0s 109us/step - loss: 0.6514 - acc: 0.6290
Epoch 5/10
3062/3062 [==============================] - 0s 104us/step - loss: 0.6490 - 

3062/3062 [==============================] - 0s 87us/step - loss: 0.6365 - acc: 0.6434
Epoch 7/10
3062/3062 [==============================] - 0s 101us/step - loss: 0.6359 - acc: 0.6342
Epoch 8/10
3062/3062 [==============================] - 0s 95us/step - loss: 0.6367 - acc: 0.6359
Epoch 9/10
3062/3062 [==============================] - 0s 110us/step - loss: 0.6358 - acc: 0.6427
Epoch 10/10
3062/3062 [==============================] - 0s 112us/step - loss: 0.6358 - acc: 0.6411
Epoch 1/10
3062/3062 [==============================] - 0s 113us/step - loss: 0.6329 - acc: 0.6381
Epoch 2/10
3062/3062 [==============================] - 0s 117us/step - loss: 0.6277 - acc: 0.6444
Epoch 3/10
3062/3062 [==============================] - 0s 99us/step - loss: 0.6268 - acc: 0.6499
Epoch 4/10
3062/3062 [==============================] - 0s 97us/step - loss: 0.6291 - acc: 0.6453
Epoch 5/10
3062/3062 [==============================] - 0s 111us/step - loss: 0.6290 - acc: 0.6463
Epoch 6/10
3062/3062 [==

With validation split, we can see that there is overfitting

In [29]:
def base_model():
    model = Sequential()
    model.add(Dense(88, kernel_initializer='glorot_uniform', kernel_regularizer=l2(0.01),
                    input_shape=(88,)))
    model.add(Dense(2, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(lr=0.01),
                  metrics=['accuracy'])
    return model;

def model_fit(model, X, y):
    model.fit(X, y,
              batch_size=batch_size,
              epochs=10,
#               validation_split=0.2,
              verbose=1)
    return model

model = deep_ensemble(base_model, model_fit, bc)

fitting sample
(10, 3062, 88) (10, 3062, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 88)                7832      
_________________________________________________________________
dense_30 (Dense)             (None, 2)                 178       
Total params: 8,010
Trainable params: 8,010
Non-trainable params: 0
_________________________________________________________________
training
Epoch 1/10
3062/3062 [==============================] - 1s 217us/step - loss: 1.7448 - acc: 0.5438
Epoch 2/10
3062/3062 [==============================] - 0s 115us/step - loss: 1.6180 - acc: 0.5813
Epoch 3/10
3062/3062 [==============================] - 0s 110us/step - loss: 1.5726 - acc: 0.5960
Epoch 4/10
3062/3062 [==============================] - 0s 113us/step - loss: 1.5426 - acc: 0.6029
Epoch 5/10
3062/3062 [==============================] - 0s 110us/step - loss: 1.5200 - 

3062/3062 [==============================] - 0s 116us/step - loss: 1.1171 - acc: 0.6463
Epoch 8/10
3062/3062 [==============================] - 0s 109us/step - loss: 1.1132 - acc: 0.6457
Epoch 9/10
3062/3062 [==============================] - 0s 109us/step - loss: 1.1090 - acc: 0.6457
Epoch 10/10
3062/3062 [==============================] - 0s 101us/step - loss: 1.1051 - acc: 0.6457
Epoch 1/10
3062/3062 [==============================] - 0s 119us/step - loss: 1.0987 - acc: 0.6444
Epoch 2/10
3062/3062 [==============================] - 0s 119us/step - loss: 1.0929 - acc: 0.6496
Epoch 3/10
3062/3062 [==============================] - 0s 113us/step - loss: 1.0878 - acc: 0.6483
Epoch 4/10
3062/3062 [==============================] - 0s 113us/step - loss: 1.0833 - acc: 0.6545
Epoch 5/10
3062/3062 [==============================] - 0s 110us/step - loss: 1.0792 - acc: 0.6564
Epoch 6/10
3062/3062 [==============================] - 0s 107us/step - loss: 1.0753 - acc: 0.6571
Epoch 7/10
3062/3062

In [43]:
def base_model():
    model = Sequential()
    model.add(Dense(88, activation='tanh',
                    kernel_initializer='glorot_uniform', kernel_regularizer=l1(0.01),
                    input_shape=(88,)))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(2, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(lr=0.01),
                  metrics=['accuracy'])
    return model;

def model_fit(model, X, y):
    model.fit(X, y,
              batch_size=batch_size,
              epochs=10,
              validation_split=0.2,
              verbose=1)
    return model

smote = SMOTE(ratio={0: 10000, 1: 10000}, n_jobs=-1, random_state=KFOLD_SEED)
model = deep_ensemble(base_model, model_fit, bc, smote)

fitting sample
(10, 3062, 88) (10, 3062)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 88)                7832      
_________________________________________________________________
dense_53 (Dense)             (None, 64)                5696      
_________________________________________________________________
dense_54 (Dense)             (None, 2)                 130       
Total params: 13,658
Trainable params: 13,658
Non-trainable params: 0
_________________________________________________________________
training


/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/imblearn/utils/validation.py:224: UserWarning: After over-sampling, the number of samples (10000) in class 0 will be larger than the number of samples in the majority class (class #0 -> 1531)
  n_samples_majority))
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/imblearn/utils/validation.py:224: UserWarning: After over-sampling, the number of samples (10000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 1531)
  n_samples_majority))


Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 3s 176us/step - loss: 5.1488 - acc: 0.6694 - val_loss: 2.8621 - val_acc: 0.3345
Epoch 2/10
16000/16000 [==============================] - 2s 133us/step - loss: 1.4709 - acc: 0.6796 - val_loss: 1.3298 - val_acc: 0.2157
Epoch 3/10
16000/16000 [==============================] - 2s 133us/step - loss: 0.9266 - acc: 0.6662 - val_loss: 1.1430 - val_acc: 0.3118
Epoch 4/10
16000/16000 [==============================] - 2s 132us/step - loss: 0.8296 - acc: 0.6687 - val_loss: 1.1511 - val_acc: 0.2125
Epoch 5/10
16000/16000 [==============================] - 2s 125us/step - loss: 0.7653 - acc: 0.6709 - val_loss: 1.1014 - val_acc: 0.1792
Epoch 6/10
16000/16000 [==============================] - 2s 132us/step - loss: 0.7209 - acc: 0.6743 - val_loss: 1.0186 - val_acc: 0.2998
Epoch 7/10
16000/16000 [==============================] - 2s 128us/step - loss: 0.6906 - acc: 0.6758 - val_loss: 0.9412 - v

/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/imblearn/utils/validation.py:224: UserWarning: After over-sampling, the number of samples (10000) in class 0 will be larger than the number of samples in the majority class (class #0 -> 1531)
  n_samples_majority))
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/imblearn/utils/validation.py:224: UserWarning: After over-sampling, the number of samples (10000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 1531)
  n_samples_majority))


Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 2s 136us/step - loss: 0.6316 - acc: 0.6866 - val_loss: 0.8913 - val_acc: 0.4298
Epoch 2/10
16000/16000 [==============================] - 2s 136us/step - loss: 0.6265 - acc: 0.6936 - val_loss: 0.9187 - val_acc: 0.3733
Epoch 3/10
16000/16000 [==============================] - 2s 134us/step - loss: 0.6236 - acc: 0.6901 - val_loss: 1.0108 - val_acc: 0.2963
Epoch 4/10
16000/16000 [==============================] - 2s 130us/step - loss: 0.6213 - acc: 0.6936 - val_loss: 0.9105 - val_acc: 0.3367
Epoch 5/10
16000/16000 [==============================] - 2s 136us/step - loss: 0.6204 - acc: 0.6921 - val_loss: 0.8435 - val_acc: 0.4555
Epoch 6/10
16000/16000 [==============================] - 2s 132us/step - loss: 0.6185 - acc: 0.6926 - val_loss: 0.8920 - val_acc: 0.3950
Epoch 7/10
16000/16000 [==============================] - 2s 134us/step - loss: 0.6185 - acc: 0.6922 - val_loss: 0.8612 - v

...

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 2s 127us/step - loss: 0.6259 - acc: 0.6743 - val_loss: 0.9682 - val_acc: 0.3053
Epoch 2/10
16000/16000 [==============================] - 2s 133us/step - loss: 0.6250 - acc: 0.6766 - val_loss: 0.8207 - val_acc: 0.4728
Epoch 3/10
16000/16000 [==============================] - 2s 141us/step - loss: 0.6256 - acc: 0.6776 - val_loss: 0.8926 - val_acc: 0.3927
Epoch 4/10
16000/16000 [==============================] - 2s 129us/step - loss: 0.6260 - acc: 0.6769 - val_loss: 0.9035 - val_acc: 0.4010
Epoch 5/10
16000/16000 [==============================] - 2s 123us/step - loss: 0.6257 - acc: 0.6746 - val_loss: 0.9933 - val_acc: 0.2840
Epoch 6/10
16000/16000 [==============================] - 2s 130us/step - loss: 0.6258 - acc: 0.6744 - val_loss: 0.9905 - val_acc: 0.3598
Epoch 7/10
16000/16000 [==============================] - 2s 132us/step - loss: 0.6253 - acc: 0.6748 - val_loss: 0.9063 - v

/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/imblearn/utils/validation.py:224: UserWarning: After over-sampling, the number of samples (10000) in class 0 will be larger than the number of samples in the majority class (class #0 -> 1531)
  n_samples_majority))
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/imblearn/utils/validation.py:224: UserWarning: After over-sampling, the number of samples (10000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 1531)
  n_samples_majority))


Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 2s 128us/step - loss: 0.6156 - acc: 0.6909 - val_loss: 0.8732 - val_acc: 0.3992
Epoch 2/10
16000/16000 [==============================] - 2s 128us/step - loss: 0.6146 - acc: 0.6905 - val_loss: 0.9288 - val_acc: 0.3992
Epoch 3/10
16000/16000 [==============================] - 2s 122us/step - loss: 0.6144 - acc: 0.6961 - val_loss: 0.7882 - val_acc: 0.5058
Epoch 4/10
16000/16000 [==============================] - 2s 129us/step - loss: 0.6146 - acc: 0.6955 - val_loss: 0.8594 - val_acc: 0.4368
Epoch 5/10
16000/16000 [==============================] - 2s 133us/step - loss: 0.6145 - acc: 0.6974 - val_loss: 0.8080 - val_acc: 0.5068
Epoch 6/10
16000/16000 [==============================] - 2s 130us/step - loss: 0.6139 - acc: 0.6951 - val_loss: 0.9544 - val_acc: 0.4047
Epoch 7/10
16000/16000 [==============================] - 2s 120us/step - loss: 0.6142 - acc: 0.6947 - val_loss: 0.9006 - v

/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/imblearn/utils/validation.py:224: UserWarning: After over-sampling, the number of samples (10000) in class 0 will be larger than the number of samples in the majority class (class #0 -> 1531)
  n_samples_majority))
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/imblearn/utils/validation.py:224: UserWarning: After over-sampling, the number of samples (10000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 1531)
  n_samples_majority))


Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 2s 130us/step - loss: 0.6215 - acc: 0.6816 - val_loss: 0.9032 - val_acc: 0.3430
Epoch 2/10
16000/16000 [==============================] - 2s 124us/step - loss: 0.6197 - acc: 0.6820 - val_loss: 0.9286 - val_acc: 0.3420
Epoch 3/10
16000/16000 [==============================] - 2s 123us/step - loss: 0.6203 - acc: 0.6809 - val_loss: 0.9270 - val_acc: 0.3780
Epoch 4/10
16000/16000 [==============================] - 2s 125us/step - loss: 0.6207 - acc: 0.6801 - val_loss: 0.9801 - val_acc: 0.2717
Epoch 5/10
16000/16000 [==============================] - 2s 122us/step - loss: 0.6203 - acc: 0.6811 - val_loss: 0.9291 - val_acc: 0.4002
Epoch 6/10
16000/16000 [==============================] - 2s 129us/step - loss: 0.6205 - acc: 0.6781 - val_loss: 0.8839 - val_acc: 0.3743
Epoch 7/10
16000/16000 [==============================] - 2s 128us/step - loss: 0.6199 - acc: 0.6835 - val_loss: 0.9578 - v